# Tutorial 8B: Data Normalisation and Transformation

When one or more datasets affects results disproportionately, normalization or scaling gives a level playing field. In this activity, we will apply different data methods for data normalisation and transformation. We first read the dataset that is used for the first part of the analysis. 


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv( 'wine_data.csv',
     header = None,
     usecols = [0,1,2]
    )

df.columns=['Class label', 'Alcohol', 'Malic acid']

df.head()

In [ ]:
df.describe()

In [ ]:
# As we can see in the tables above, the features, Alcohol (percent/volumne) and Malic acid (g/l) 
# are measured on different scales, so scaling is necessary prior to any comparison or combination of data.
df.Alcohol.mean() / df["Malic acid"].mean() # difference is factor of ~5x


## Z-Score Normalisation (standardisation): 

We use sklearn linrary for standardise data (mean=0, SD=1). 

In [ ]:
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(df[['Alcohol', 'Malic acid']])
df_std = std_scale.transform(df[['Alcohol', 'Malic acid']]) # an array not a df

In [ ]:
std_scale.fit # defaults mean & std

In [ ]:
df_std[0:5] # the scaled data (some of it)

In [ ]:
# put it alongside data... to view
df['Ascaled'] = df_std[:,0] # so 'Ascaled' is Alcohol scaled

In [ ]:
df['MAscaled'] = df_std[:,1] # and 'MAscaled' is Malic acid scaled

In [ ]:
df.head()

Now, display the normalised values for both features. They have mean of 0 and SD= 1

In [ ]:
df.describe() # check that μ = 0 and σ = 1... approx

Print out values:

In [ ]:
print('Mean after standardisation:\nAlcohol = {:.2f}, Malic acid = {:.2f}'
      .format(df_std[:,0].mean(), df_std[:,1].mean()))
print('\nStandard deviation after standardisation:\nAlcohol = {:.2f}, Malic acid = {:.2f}'
      .format(df_std[:,0].std(), df_std[:,1].std()))

# Mean after standardization:
# Alcohol=0.00, Malic acid=0.00
# force those tiny little numbers (-8.619821e-16 & -8.357859e-17) to behave...
# Standard deviation after standardization:
# Alcohol=1.00, Malic acid=1.00

#### We plot first the data before normalisation. 

In [ ]:
%matplotlib inline
df["Alcohol"].plot(), df["Malic acid"].plot()

#### Now, we plot  the standardised data 
Observe the range and the centre of the distribution for the standardised features. 

In [ ]:
# or split them from the others
df["MAscaled"].plot(), df["Ascaled"].plot()


You can see from above graphs that both original and standardised data are in the same shape but shifted. What is the color of the normalised line?


In [ ]:
df["Ascaled"].plot(), df["Alcohol"].plot()


In [ ]:
df["MAscaled"].plot(), df["Malic acid"].plot()


#  MinMax Noramlisation:

In this section, we discuss a different type of normalisation for reshaping the range of data. We process the same data we used in the previous section. We can implement this eith either sklearn or manually. 

#### * Using sklearn:

In [ ]:
minmax_scale = preprocessing.MinMaxScaler().fit(df[['Alcohol', 'Malic acid']])
df_minmax = minmax_scale.transform(df[['Alcohol', 'Malic acid']])

In [ ]:
df_minmax[0:5]

### *Manually:

In [ ]:
# DIY MinMax, be careful with -ves, may need abs
# df.Alcohol.describe()
minA = df.Alcohol.min()
maxA = df.Alcohol.max()
a = df.Alcohol[0] # the first value, for practice
a, minA, maxA # so our test subject is much nearer max, it should scale to around 0.9

In [ ]:
mma = (a - minA) / (maxA - minA)
mma # and compare with the previous method

In [ ]:
df_minmax[0][0] # same

In [ ]:
df[df.Alcohol == df.Alcohol.max()] # where is max?

In [ ]:
a = df.Alcohol[8]
a

In [ ]:
mma = (a - minA) / (maxA - minA)
mma # max should map to 1.0 exactly (same for min)

In [ ]:
print('Min-value after min-max scaling:\nAlcohol = {:.2f}, Malic acid = {:.2f}'
      .format(df_minmax[:,0].min(), df_minmax[:,1].min()))
print('\nMax-value after min-max scaling:\nAlcohol = {:.2f}, Malic acid = {:.2f}'
      .format(df_minmax[:,0].max(), df_minmax[:,1].max()))
# Min-value after min-max scaling:
# Alcohol=0.00, Malic acid=0.00

# Max-value after min-max scaling:
# Alcohol=1.00, Malic acid=1.00

### We now plot the original, standardised and normalised data values. 

In [ ]:
# and plot
%matplotlib inline

from matplotlib import pyplot as plt

def plot():
    plt.figure(figsize=(8,6))

    plt.scatter(df['Alcohol'], df['Malic acid'],
            color='green', label='input scale', alpha=0.5)

 #   plt.scatter(df_std[:,0], df_std[:,1], color='red',
 #           label='Standardized [$$N  (\mu=0, \; \sigma=1)$$]', alpha=0.3)
    plt.scatter(df_std[:,0], df_std[:,1], color='red',
             label='Standardized u=0, s=1', alpha=0.3) # can't print: μ = 0, σ = 0
    
    plt.scatter(df_minmax[:,0], df_minmax[:,1],
            color='blue', label='min-max scaled [min=0, max=1]', alpha=0.3)

    plt.title('Alcohol and Malic Acid content of the wine dataset')
    plt.xlabel('Alcohol')
    plt.ylabel('Malic Acid')
    plt.legend(loc='upper left')
    plt.grid()

    plt.tight_layout()

plot()
plt.show()

#### The plot above includes the wine datapoints on all three different scales: 
* the input scale  where the alcohol content was measured in volume-percent (green),
* the standardized features (red), and 
* the normalized features (blue). 

#### In the following plot, we will zoom in into the three different axis-scales while dispalying class values. 

In [ ]:
fig, ax = plt.subplots(3, figsize=(6,14))

for a,d,l in zip(range(len(ax)),
               (df[['Alcohol', 'Malic acid']].values, df_std, df_minmax),
               ('Input scale',
                'Standardized [u=0 s=1]',
                'min-max scaled [min=0, max=1]')
                ):
    for i,c in zip(range(1,4), ('red', 'blue', 'green')):
        ax[a].scatter(d[df['Class label'].values == i, 0],
                  d[df['Class label'].values == i, 1],
                  alpha=0.5,
                  color=c,
                  label='Class %s' %i
                  )
    ax[a].set_title(l)
    ax[a].set_xlabel('Alcohol')
    ax[a].set_ylabel('Malic Acid')
    ax[a].legend(loc='upper left')
    ax[a].grid()

plt.tight_layout()

plt.show()

## Using other libraries for normalisation:

We use a simple synthetic data (x) to explain how can we implement normalisation manually and with numpy. We strat with z-score normalisation. 

### Z-score normalisation:

In [ ]:
x = [1,4,5,6,6,2,3]
mean = sum(x)/len(x)
std_dev = (1/len(x) * sum([ (x_i - mean)**2 for x_i in x]))**0.5
mean, std_dev, np.mean(x), np.std(x)

In [ ]:
# 3 & 0 no good, look:
z_scores = [(x_i - mean)/std_dev for x_i in x]

In [ ]:
mean = sum(x)/float(len(x)) # force float, other ways to do this, e.g. future, numpy, Py3?
mean

In [ ]:
std_dev = (1/float(len(x)) * sum([(x_i - mean)**2 for x_i in x]))**0.5
std_dev

In [ ]:
# or
np.std(x)

In [ ]:
z_scores = [(x_i - mean)/std_dev for x_i in x]
z_scores

### Min-Max Normalisation:

In [ ]:
# Min-Max scaling
minmax = [(x_i - min(x)) / (max(x) - min(x)) for x_i in x]
minmax

In [ ]:
# that's also wrong, min (1) should scale to 0, max (6) to 1
# with the others distributed between [0 to 1] 
# Min-Max scaling, fix
minx = min(x) # and make it faster too
maxx = max(x) 
diff = float(maxx - minx) # force one to be float

minmax = [(x_i - minx) / diff for x_i in x]
minmax

### Using numpy:

In [ ]:
# numpy Standardization

x_np = np.asarray(x) # convert to list to array, better for maths 
z_scores_np = (x_np - x_np.mean()) / x_np.std()

# Min-Max scaling

np_minmax = (x_np - x_np.min()) / (x_np.max() - x_np.min())

In [ ]:
np_minmax # also wrong

In [ ]:
np_minmax = (x_np - x_np.min()) / (float(x_np.max()) - x_np.min()) # force float

In [ ]:
z_scores_np

In [ ]:
np_minmax

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10,5))

y_pos = [0 for i in range(len(x))]

ax1.scatter(z_scores, y_pos, color='g')
ax1.set_title('Python standardization', color='g')

ax2.scatter(minmax, y_pos, color='g')
ax2.set_title('Python Min-Max scaling', color='g')

ax3.scatter(z_scores_np, y_pos, color='b')
ax3.set_title('Python NumPy standardization', color='b')

ax4.scatter(np_minmax, y_pos, color='b')
ax4.set_title('Python NumPy Min-Max scaling', color='b')

plt.tight_layout()

for ax in (ax1, ax2, ax3, ax4):
    ax.get_yaxis().set_visible(False)
    ax.grid()

plt.show()

In [ ]:
# and we can see the effects of scaling using z-score (on the left), and MinMax (on the right)
# (except that there are two 1s, from the two 6s) e.g.
plt.hist(minmax)

In [ ]:
# and you can see Min-Max is standardised to [0.0....1.0] by default, 
# you can change by using a range, e.g. want range to be 3 to 5 so: 
np_minmax2 = (x_np - x_np.min()) / (float(x_np.max()) - x_np.min()) * (5 - 3) + 3

In [ ]:
np_minmax2

# Decimal scaling 
In decimal scaling, we need to find the abs max then divide all numbers by that number rounded up i.e. 6 --> 10


In [ ]:
abs(max(x))

In [ ]:
# then how many digits?
# use round if float
len(str(abs(max(x))))

In [ ]:
# then force up to next digit & force float
float(10**len(str(abs(max(x)))))

In [ ]:
# then divide all by this, works for -ves?
ds = x_np / 10.0
ds

In [ ]:
# and you can see values are scaled from 0.1 to 0.6
np.round(6,-1) # another way to do it

In [ ]:
plt.scatter(ds, y_pos) 

# Data Transformation:

Another way to reshape data is to perform data transformation. We will display an example of data that is with right skew (positive skew). We will need to compress large values. We first read the data used for this activity. 

In [ ]:
import pandas as pd
data = pd.read_csv("bmr.csv")

In [ ]:
data.head()

In [ ]:
plt.scatter(data["BMR(W)"], data["Mass(g)"]) # before

### So, which transformation type will suit this data?

In Tukey's ladder of power, we discussed different kind of transformation. We will try some here:

### Root transformation:

In [ ]:
import math
data['lmr'] = None
i = 0
for row in data.iterrows():
    data['lmr'].at[i] = math.sqrt(data["BMR(W)"][i])
    i += 1

    
data.head()

In [ ]:
data['lbm'] = None
i = 0
for row in data.iterrows():
    data['lbm'].at[i] = math.sqrt(data["Mass(g)"][i])
    i += 1

    
data.head()

In [ ]:
plt.scatter(data.lbm, data.lmr) # and after

#### Does it give a better spread of the data? Let's try something else.

### Square power transformation:

In [ ]:
import math
data['lmr'] = None
i = 0
for row in data.iterrows():
    data['lmr'].at[i] = math.pow(data["BMR(W)"][i],2)
    i += 1

    
data.head()

In [ ]:
data['lbm'] = None
i = 0
for row in data.iterrows():
    data['lbm'].at[i] = math.pow(data["Mass(g)"][i],2)
    i += 1

    
data.head()

In [ ]:
plt.scatter(data.lbm, data.lmr) # and after

#### Can you justify the output of this figure?

### Log transformation:

In [ ]:
import math
data['lmr'] = None
i = 0
for row in data.iterrows():
    data['lmr'].at[i] = math.log(data["BMR(W)"][i])
    i += 1

    
data.head()

In [ ]:
data['lbm'] = None
i = 0
for row in data.iterrows():
    data['lbm'].at[i] = math.log(data["Mass(g)"][i])
    i += 1

    
data.head()

In [ ]:
plt.scatter(data.lbm, data.lmr) # and after

Apparently, the best transformation for this data is log transformation. As the data is positively skewed. we will need to compress large values. That means we need to move down the ladder of powers to spread out data that is clustered at lower values. Therefore, logarithmic is the appropriate transformation in this case.

## Exercise:
Consider the following dataset:


In [ ]:
body_mass = [32000, 37800, 347000, 4200, 196500, 100000, 4290, 
32000, 65000, 69125, 9600, 133300, 150000, 407000, 115000, 67000, 
325000, 21500, 58588, 65320, 85000, 135000, 20500, 1613, 1618]

metabolic_rate = [49.984, 51.981, 306.770, 10.075, 230.073, 
148.949, 11.966, 46.414, 123.287, 106.663, 20.619, 180.150, 
200.830, 224.779, 148.940, 112.430, 286.847, 46.347, 142.863, 
106.670, 119.660, 104.150, 33.165, 4.900, 4.865]

#### What will be the appropriate transformation to apply for this data?